In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import datetime
import time
import pickle
import gc
import warnings
import requests
import gzip
from io import BytesIO
from sklearn.preprocessing import StandardScaler


In [2]:
# Limpia la memoria RAM
gc.collect()

8

In [3]:
import pandas as pd

def leer_datos(ruta_archivo):
    # Asegúrate de que la ruta del archivo y el formato sean correctos
    df = pd.read_csv(ruta_archivo, compression='gzip', sep='\t')
    return df

In [4]:


file_url = 'https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_sellout_01.txt.gz'

# Download the gzipped file using requests
response = requests.get(file_url)

# Check if the request was successful
if response.status_code == 200:
    # Create a file-like object from the response content
    file_content = BytesIO(response.content)

    # Open the gzipped file and read its lines
    with gzip.open(file_content, 'rt', encoding='utf-8') as file:
        lines = file.readlines()
    # Do further processing with the 'lines' variable

else:
    print("Failed to download the file.")


# Extract column names from the first 6 lines (assuming 6 lines for column names)
column_names = [line.strip() for line in lines[:6]]

# Create a dictionary from the data
data = {}
for i, col_name in enumerate(column_names):
    data[col_name] = [line.strip() for line in lines[6 + i::6]]

# Create the DataFrame
df_sellout = pd.DataFrame(data)

stocks = leer_datos('https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_stocks_02.txt.gz')
master_prod =  pd.read_csv('https://storage.googleapis.com/open-courses/austral2023-labo3-d0e5/tb_productos_02.txt', sep='\t')


In [5]:
df_sellout.head()

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,2,0.053,0.053
1,201701,10032,20524,1,0.13628,0.13628
2,201701,10217,20524,1,0.03028,0.03028
3,201701,10125,20524,1,0.02271,0.02271
4,201701,10012,20524,11,1.54452,1.54452


In [6]:
# No hay stocks antes del periodo 201810
pd.DataFrame(stocks['periodo'].value_counts()).sort_values(by='periodo', ascending=False)


,count
periodo,
201912,925
201911,937
201910,949
201909,929
201908,928
201907,931
201906,914
201905,909
201904,915


In [7]:
# Convertir las columnas a string, int o float antes de hacer el merge
df_sellout['periodo'] = df_sellout['periodo'].astype(int)
df_sellout['product_id'] = df_sellout['product_id'].astype(int)
df_sellout['customer_id'] = df_sellout['customer_id'].astype(int)

df_sellout[['cust_request_qty', 'cust_request_tn','tn']] = df_sellout[['cust_request_qty', 'cust_request_tn','tn']].astype(float)


stocks['periodo'] = stocks['periodo'].astype(int)
stocks['product_id'] = stocks['product_id'].astype(int)
stocks['stock_final']=stocks['stock_final'].astype(float)

master_prod['product_id'] = master_prod['product_id'].astype(int)
master_prod['cat1'] = master_prod['cat1'].astype(str)
master_prod['cat2'] = master_prod['cat2'].astype(str)
master_prod['cat3'] = master_prod['cat3'].astype(str)
master_prod['brand'] = master_prod['brand'].astype(str)


master_prod['sku_size']=master_prod['sku_size'].astype(float)


In [8]:
df_sellout_prod=df_sellout.groupby(['periodo','product_id'])[['cust_request_qty', 'cust_request_tn','tn']].sum().reset_index()

In [9]:
df_sellout_prod_client=df_sellout.groupby(['periodo','customer_id','product_id'])[['cust_request_qty', 'cust_request_tn','tn']].sum().reset_index()

In [10]:
df_sellout_prod=df_sellout_prod.sort_values(by=['product_id','periodo' ])
df_sellout_prod['tn_mas_1']=df_sellout_prod.groupby('product_id')['tn'].shift(-1)
df_sellout_prod['tn_mas_2']=df_sellout_prod.groupby('product_id')['tn'].shift(-2)
df_sellout_prod=df_sellout_prod.reset_index(drop=True)


In [11]:
df_sellout_prod_client=df_sellout_prod_client.sort_values(by=['customer_id','product_id','periodo' ])
df_sellout_prod_client['tn_mas_1']=df_sellout_prod_client.groupby(['customer_id','product_id'])['tn'].shift(-1)
df_sellout_prod_client['tn_mas_2']=df_sellout_prod_client.groupby(['customer_id','product_id'])['tn'].shift(-2)
df_sellout_prod_client=df_sellout_prod_client.reset_index(drop=True)


In [12]:
#Imputamos a 0 los valores nulos de tn_mas_1 y tn_mas_2
df_sellout_prod['tn_mas_1']=df_sellout_prod['tn_mas_1'].fillna(0)
df_sellout_prod['tn_mas_2']=df_sellout_prod['tn_mas_2'].fillna(0)
df_sellout_prod_client['tn_mas_1']=df_sellout_prod_client['tn_mas_1'].fillna(0)
df_sellout_prod_client['tn_mas_2']=df_sellout_prod_client['tn_mas_2'].fillna(0)

In [13]:
df_sellout_prod

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391
...,...,...,...,...,...,...,...
31238,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000
31239,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000
31240,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000
31241,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000


In [14]:
df_sellout_prod_client

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2
0,201701,10001,20001,11.0,99.43861,99.43861,198.84365,92.46537
1,201702,10001,20001,23.0,198.84365,198.84365,92.46537,13.29728
2,201703,10001,20001,33.0,92.46537,92.46537,13.29728,101.00563
3,201704,10001,20001,8.0,13.29728,13.29728,101.00563,128.04792
4,201705,10001,20001,15.0,101.20711,101.00563,128.04792,101.20711
...,...,...,...,...,...,...,...,...
2945813,201702,10636,20838,1.0,0.00190,0.00190,0.00000,0.00000
2945814,201702,10636,20969,1.0,0.00433,0.00433,0.00000,0.00000
2945815,201702,10636,21178,1.0,0.00197,0.00197,0.00000,0.00000
2945816,201709,10637,20657,1.0,0.00144,0.00144,0.00000,0.00000


# Vemos que el master tiene duplicados

In [15]:
duplicated_mask = master_prod.duplicated(subset=['product_id'], keep=False)
duplicated_cols = master_prod.loc[duplicated_mask, :]
duplicated_cols


,cat1,cat2,cat3,brand,sku_size,product_id
36,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20010
37,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20010
38,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20022
39,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20022
41,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20021
42,HC,ROPA LAVADO,Polvo,LIMPIEX,400.0,20021
43,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20020
44,HC,ROPA LAVADO,Polvo,LIMPIEX,800.0,20020
541,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI,2.0,20278
542,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI,2.0,20278


In [16]:
#eliminamos duplicados
master_prod.drop_duplicates(subset=['product_id'], inplace=True)


# Ahora mergeamos la base prod y la prod_customer

In [17]:
# Primero unimos df_sellout_prod con stocks
df_merge1 = pd.merge(df_sellout_prod, stocks, on=['periodo', 'product_id'], how='left')

# Luego unimos el resultado con master_prod
df_final_sellout_prod = pd.merge(df_merge1, master_prod, on='product_id', how='left')

df_final_sellout_prod['cat_1_2']=df_final_sellout_prod['cat1']+'-'+df_final_sellout_prod['cat2']
df_final_sellout_prod['cat_1_2_3']=df_final_sellout_prod['cat1']+'-'+df_final_sellout_prod['cat2']+'-'+df_final_sellout_prod['cat3']
df_final_sellout_prod['cat_1_2_3_brand']=df_final_sellout_prod['cat1']+'-'+df_final_sellout_prod['cat2']+'-'+df_final_sellout_prod['cat3']+'-'+df_final_sellout_prod['brand']

In [18]:
df_final_sellout_prod

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31239,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31240,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31241,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Primero unimos df_sellout_prod con stocks
df_merge1 = pd.merge(df_sellout_prod_client, stocks, on=['periodo', 'product_id'], how='left')

# Luego unimos el resultado con master_prod
df_final_sellout_prod_client = pd.merge(df_merge1, master_prod, on='product_id', how='left')

df_final_sellout_prod_client['cat_1_2']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']
df_final_sellout_prod_client['cat_1_2_3']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']+'-'+df_final_sellout_prod_client['cat3']
df_final_sellout_prod_client['cat_1_2_3_brand']=df_final_sellout_prod_client['cat1']+'-'+df_final_sellout_prod_client['cat2']+'-'+df_final_sellout_prod_client['cat3']+'-'+df_final_sellout_prod_client['brand']

In [20]:
#df_test = df_final[(df_final['periodo'] >= 201810) & (df_final['periodo'] <= 201902)]
#df_test=df_test.reset_index(drop=True)


## Hasta aca tenemos dos bases, una que esta anivel de periodo y product_id y una que esta a nivel de periodo, product_id y customer_id.

In [21]:
# Periodo y producto
df_final_sellout_prod

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31239,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31240,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31241,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Periodo, cliente y producto
df_final_sellout_prod_client

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand
0,201701,10001,20001,11.0,99.43861,99.43861,198.84365,92.46537,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
1,201702,10001,20001,23.0,198.84365,198.84365,92.46537,13.29728,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
2,201703,10001,20001,33.0,92.46537,92.46537,13.29728,101.00563,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
3,201704,10001,20001,8.0,13.29728,13.29728,101.00563,128.04792,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
4,201705,10001,20001,15.0,101.20711,101.00563,128.04792,101.20711,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945813,201702,10636,20838,1.0,0.00190,0.00190,0.00000,0.00000,NaN,PC,DEOS,Aero,DEOS2,60.0,PC-DEOS,PC-DEOS-Aero,PC-DEOS-Aero-DEOS2
2945814,201702,10636,20969,1.0,0.00433,0.00433,0.00000,0.00000,NaN,PC,CABELLO,Tratamiento Fem,NIVEA,200.0,PC-CABELLO,PC-CABELLO-Tratamiento Fem,PC-CABELLO-Tratamiento Fem-NIVEA
2945815,201702,10636,21178,1.0,0.00197,0.00197,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2945816,201709,10637,20657,1.0,0.00144,0.00144,0.00000,0.00000,NaN,HC,HOGAR,Repelente,OFF,165.0,HC-HOGAR,HC-HOGAR-Repelente,HC-HOGAR-Repelente-OFF


## División de train y test

In [23]:
mes_train=201902
# Mes de predicción: mes_train+2
mes_predict=201904

In [24]:
df_final_sellout_prod_predict=df_final_sellout_prod[df_final_sellout_prod['periodo']>mes_train].reset_index(drop=True)
df_final_sellout_prod=df_final_sellout_prod[df_final_sellout_prod.periodo<=mes_train].reset_index(drop=True)

In [25]:
df_final_sellout_prod_client_predict=df_final_sellout_prod_client[df_final_sellout_prod_client['periodo']>mes_train].reset_index(drop=True)
df_final_sellout_prod_client=df_final_sellout_prod_client[df_final_sellout_prod_client.periodo<=mes_train].reset_index(drop=True)

In [26]:
# Revisar si los product_id de la base df_final_sellout_prod_client están en df_final_sellout_prod_client_predict
if set(df_final_sellout_prod_client.product_id.unique()).issubset(set(df_final_sellout_prod_client_predict[df_final_sellout_prod_client_predict.periodo==201904].product_id.unique())):
    print("Todos los product_id de df_final_sellout_prod_client están en df_final_sellout_prod_client_predict.")
else:
    # Encontrar los product_id que están en df_final_sellout_prod_client_predict pero no en df_final_sellout_prod_client
    missing_product_ids = set(df_final_sellout_prod_client_predict[df_final_sellout_prod_client_predict.periodo==201904].product_id.unique()) - set(df_final_sellout_prod_client.product_id.unique())
    print("Los siguientes product_id están en df_final_sellout_prod_client_predict pero no en df_final_sellout_prod_client:")
    print(missing_product_ids)


Los siguientes product_id están en df_final_sellout_prod_client_predict pero no en df_final_sellout_prod_client:
{20355, 21252, 21128, 20746, 20491, 20620, 20623, 21265, 21266, 21267, 20884, 20757, 21269, 20633, 21276, 20510, 20521, 21033, 20907, 20525, 20526, 20785, 20531, 20659, 20920, 20537, 20795, 20668, 21179, 20286, 20414, 20548, 20681, 21196, 21073, 20822, 20442, 21086, 20575, 20577, 21092, 20966, 20458, 21228, 20719, 20593, 20859, 20987, 20603, 20476}


In [27]:
df_final_sellout_prod

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Estandarizacion de variables

### Estandarizamos tn por product_id

In [28]:
# Define the column you want to standardize
column_to_standardize = 'tn'

# Group by 'product_id'
grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

# Initialize a dictionary to store scalers for each 'product_id'
scalers = {}

# Create an empty DataFrame to store the standardized values
standardized_df = pd.DataFrame()

# Standardize each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be standardized
    values_to_standardize = group_data.values.reshape(-1, 1)

    # Initialize and fit the scaler on the values without transforming
    scaler = StandardScaler()
    scaler.fit(values_to_standardize)

    # Save the scaler for this group in the dictionary
    scalers[group_name] = scaler

    # Transform the data
    standardized_values = scaler.transform(values_to_standardize)

    # Create a DataFrame with the standardized values and 'product_id' as the index
    group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall standardized DataFrame
    standardized_df = pd.concat([standardized_df, group_df])

# Sort the DataFrame by index to get the original order
standardized_df = standardized_df.sort_index()

# Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

# Save the scalers dictionary to a file
import joblib

joblib.dump(scalers, 'scalers_product_tn.joblib')

# ...

# Load the scalers dictionary from the file
scalers = joblib.load('scalers_product_tn.joblib')

# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
inverse_transformed_df = inverse_transformed_df.sort_index()




scalers = joblib.load('scalers_product_tn.joblib')
scalers

def scale_columns(df, columns_to_scale, scaler_dict):
    scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be scaled for each column separately
            scaled_values = {}
            for column in columns_to_scale:
                values_to_scale = group_data[column].values.reshape(-1, 1)
                
                # Scale the data
                scaled_column = scaler.transform(values_to_scale)
                
                # Store the scaled column in the dictionary
                scaled_values[column] = scaled_column.flatten()

            # Create a DataFrame with the scaled values and 'product_id' as the index
            group_df = pd.DataFrame(scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall scaled DataFrame
            scaled_df = pd.concat([scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    scaled_df = scaled_df.sort_index()

    return scaled_df

# Example usage:
# (Assuming you have already loaded the 'scalers' dictionary)

# Columns to scale
columns_to_scale = ['tn', 'tn_mas_1', 'tn_mas_2']

# Call the function to scale the specified columns
scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

# Print the resulting DataFrame
print(scaled_columns_df)


# mergeamos las variables estandarizadas

df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

df_final_sellout_prod


             tn  tn_mas_1  tn_mas_2
0     -1.334149 -1.760963 -0.183798
1     -1.760963 -0.183798 -0.912226
2     -0.183798 -0.912226  0.436790
3     -0.912226  0.436790  0.492543
4      0.436790  0.492543 -1.034841
...         ...       ...       ...
21988  0.000000 -0.006990 -0.006990
21989  0.000000 -0.006510 -0.006510
21990  0.000000 -0.005790 -0.005790
21991  0.000000 -0.005730 -0.005730
21992  0.000000 -0.005460 -0.005460

[21993 rows x 3 columns]


,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand,tn_scaled,tn_mas_1_scaled,tn_mas_2_scaled
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.334149,-1.760963,-0.183798
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.760963,-0.183798,-0.912226
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.183798,-0.912226,0.436790
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.912226,0.436790,0.492543
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,0.436790,0.492543,-1.034841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006990,-0.006990
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006510,-0.006510
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005790,-0.005790
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005730,-0.005730


### Probamos invertir la transformacion

In [29]:
scalers = joblib.load('scalers_product_tn.joblib')
def inverse_scale_columns(df, columns_to_inverse_scale, scaler_dict):
    inverse_scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the original DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the original DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be inverse scaled for each column separately
            inverse_scaled_values = {}
            for column in columns_to_inverse_scale:
                values_to_inverse_scale = group_data[column].values.reshape(-1, 1)

                # Inverse scale the data
                inverse_scaled_column = scaler.inverse_transform(values_to_inverse_scale)

                # Store the inverse scaled column in the dictionary
                inverse_scaled_values[column] = inverse_scaled_column.flatten()

            # Create a DataFrame with the inverse scaled values and 'product_id' as the index
            group_df = pd.DataFrame(inverse_scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall inverse scaled DataFrame
            inverse_scaled_df = pd.concat([inverse_scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_scaled_df = inverse_scaled_df.sort_index()

    return inverse_scaled_df

# Columns to inverse scale
columns_to_inverse_scale = ['tn_scaled', 'tn_mas_1_scaled', 'tn_mas_2_scaled']

# Call the function to inverse scale the specified columns
inverse_scaled_columns_df = inverse_scale_columns(df_final_sellout_prod, columns_to_inverse_scale, scalers)

# Print the resulting DataFrame
print(inverse_scaled_columns_df)

        tn_scaled  tn_mas_1_scaled  tn_mas_2_scaled
0       934.77222        798.01620       1303.35771
1       798.01620       1303.35771       1069.96130
2      1303.35771       1069.96130       1502.20132
3      1069.96130       1502.20132       1520.06539
4      1502.20132       1520.06539       1030.67391
...           ...              ...              ...
21988     0.00699          0.00000          0.00000
21989     0.00651          0.00000          0.00000
21990     0.00579          0.00000          0.00000
21991     0.00573          0.00000          0.00000
21992     0.00546          0.00000          0.00000

[21993 rows x 3 columns]


### Estandarizamos cust_request_qty por product_id

In [30]:
# Define the column you want to standardize
column_to_standardize = 'cust_request_qty'

# Group by 'product_id'
grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

# Initialize a dictionary to store scalers for each 'product_id'
scalers = {}

# Create an empty DataFrame to store the standardized values
standardized_df = pd.DataFrame()

# Standardize each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be standardized
    values_to_standardize = group_data.values.reshape(-1, 1)

    # Initialize and fit the scaler on the values without transforming
    scaler = StandardScaler()
    scaler.fit(values_to_standardize)

    # Save the scaler for this group in the dictionary
    scalers[group_name] = scaler

    # Transform the data
    standardized_values = scaler.transform(values_to_standardize)

    # Create a DataFrame with the standardized values and 'product_id' as the index
    group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall standardized DataFrame
    standardized_df = pd.concat([standardized_df, group_df])

# Sort the DataFrame by index to get the original order
standardized_df = standardized_df.sort_index()

# Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

# Save the scalers dictionary to a file
import joblib

joblib.dump(scalers, 'scalers_product_request_qty.joblib')

# ...

# Load the scalers dictionary from the file
scalers = joblib.load('scalers_product_request_qty.joblib')

# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
inverse_transformed_df = inverse_transformed_df.sort_index()




scalers = joblib.load('scalers_product_request_qty.joblib')
scalers

def scale_columns(df, columns_to_scale, scaler_dict):
    scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be scaled for each column separately
            scaled_values = {}
            for column in columns_to_scale:
                values_to_scale = group_data[column].values.reshape(-1, 1)
                
                # Scale the data
                scaled_column = scaler.transform(values_to_scale)
                
                # Store the scaled column in the dictionary
                scaled_values[column] = scaled_column.flatten()

            # Create a DataFrame with the scaled values and 'product_id' as the index
            group_df = pd.DataFrame(scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall scaled DataFrame
            scaled_df = pd.concat([scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    scaled_df = scaled_df.sort_index()

    return scaled_df

# Example usage:
# (Assuming you have already loaded the 'scalers' dictionary)

# Columns to scale
columns_to_scale = ['cust_request_qty']

# Call the function to scale the specified columns
scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

# Print the resulting DataFrame
print(scaled_columns_df)


# mergeamos las variables estandarizadas

df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

df_final_sellout_prod

       cust_request_qty
0              0.279719
1             -0.184076
2              0.575758
3             -1.693875
4              2.470409
...                 ...
21988          0.000000
21989          0.000000
21990          0.000000
21991          0.000000
21992          0.000000

[21993 rows x 1 columns]


,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,cat3,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand,tn_scaled,tn_mas_1_scaled,tn_mas_2_scaled,cust_request_qty_scaled
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.334149,-1.760963,-0.183798,0.279719
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.760963,-0.183798,-0.912226,-0.184076
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.183798,-0.912226,0.436790,0.575758
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.912226,0.436790,0.492543,-1.693875
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,0.436790,0.492543,-1.034841,2.470409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006990,-0.006990,0.000000
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006510,-0.006510,0.000000
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005790,-0.005790,0.000000
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005730,-0.005730,0.000000


### Probamos invirtiendo la estandarización

In [31]:
scalers = joblib.load('scalers_product_request_qty.joblib')
def inverse_scale_columns(df, columns_to_inverse_scale, scaler_dict):
    inverse_scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the original DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the original DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be inverse scaled for each column separately
            inverse_scaled_values = {}
            for column in columns_to_inverse_scale:
                values_to_inverse_scale = group_data[column].values.reshape(-1, 1)

                # Inverse scale the data
                inverse_scaled_column = scaler.inverse_transform(values_to_inverse_scale)

                # Store the inverse scaled column in the dictionary
                inverse_scaled_values[column] = inverse_scaled_column.flatten()

            # Create a DataFrame with the inverse scaled values and 'product_id' as the index
            group_df = pd.DataFrame(inverse_scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall inverse scaled DataFrame
            inverse_scaled_df = pd.concat([inverse_scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_scaled_df = inverse_scaled_df.sort_index()

    return inverse_scaled_df

# Columns to inverse scale
columns_to_inverse_scale = ['cust_request_qty_scaled']

# Call the function to inverse scale the specified columns
inverse_scaled_columns_df = inverse_scale_columns(df_final_sellout_prod, columns_to_inverse_scale, scalers)

# Print the resulting DataFrame
print(inverse_scaled_columns_df)

       cust_request_qty_scaled
0                        479.0
1                        432.0
2                        509.0
3                        279.0
4                        701.0
...                        ...
21988                      1.0
21989                      1.0
21990                      1.0
21991                      1.0
21992                      1.0

[21993 rows x 1 columns]


### Estandarizamos cust_request_tn por product_id

In [32]:
# Define the column you want to standardize
column_to_standardize = 'cust_request_tn'

# Group by 'product_id'
grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

# Initialize a dictionary to store scalers for each 'product_id'
scalers = {}

# Create an empty DataFrame to store the standardized values
standardized_df = pd.DataFrame()

# Standardize each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be standardized
    values_to_standardize = group_data.values.reshape(-1, 1)

    # Initialize and fit the scaler on the values without transforming
    scaler = StandardScaler()
    scaler.fit(values_to_standardize)

    # Save the scaler for this group in the dictionary
    scalers[group_name] = scaler

    # Transform the data
    standardized_values = scaler.transform(values_to_standardize)

    # Create a DataFrame with the standardized values and 'product_id' as the index
    group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall standardized DataFrame
    standardized_df = pd.concat([standardized_df, group_df])

# Sort the DataFrame by index to get the original order
standardized_df = standardized_df.sort_index()

# Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

# Save the scalers dictionary to a file
import joblib

joblib.dump(scalers, 'scalers_product_request_tn.joblib')

# ...

# Load the scalers dictionary from the file
scalers = joblib.load('scalers_product_request_tn.joblib')

# Create an empty DataFrame to store the inverse transformed values
inverse_transformed_df = pd.DataFrame()

# Inverse transform each group and concatenate the results
for group_name, group_data in grouped:
    # Extract the values to be inverse transformed
    values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

    # Retrieve the scaler for this group from the dictionary
    scaler = scalers[group_name]

    # Inverse transform the data using the original mean and standard deviation
    inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

    # Create a DataFrame with the inverse transformed values and 'product_id' as the index
    group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

    # Concatenate the group DataFrame to the overall inverse transformed DataFrame
    inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

# Sort the DataFrame by index to get the original order
inverse_transformed_df = inverse_transformed_df.sort_index()




scalers = joblib.load('scalers_product_request_tn.joblib')
scalers

def scale_columns(df, columns_to_scale, scaler_dict):
    scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be scaled for each column separately
            scaled_values = {}
            for column in columns_to_scale:
                values_to_scale = group_data[column].values.reshape(-1, 1)
                
                # Scale the data
                scaled_column = scaler.transform(values_to_scale)
                
                # Store the scaled column in the dictionary
                scaled_values[column] = scaled_column.flatten()

            # Create a DataFrame with the scaled values and 'product_id' as the index
            group_df = pd.DataFrame(scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall scaled DataFrame
            scaled_df = pd.concat([scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    scaled_df = scaled_df.sort_index()

    return scaled_df

# Example usage:
# (Assuming you have already loaded the 'scalers' dictionary)

# Columns to scale
columns_to_scale = ['cust_request_tn']

# Call the function to scale the specified columns
scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

# Print the resulting DataFrame
print(scaled_columns_df)


# mergeamos las variables estandarizadas

df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

df_final_sellout_prod

       cust_request_tn
0            -1.440110
1            -1.749077
2            -0.272573
3            -0.860182
4             0.380806
...                ...
21988         0.000000
21989         0.000000
21990         0.000000
21991         0.000000
21992         0.000000

[21993 rows x 1 columns]


,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,...,brand,sku_size,cat_1_2,cat_1_2_3,cat_1_2_3_brand,tn_scaled,tn_mas_1_scaled,tn_mas_2_scaled,cust_request_qty_scaled,cust_request_tn_scaled
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,...,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.334149,-1.760963,-0.183798,0.279719,-1.440110
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,...,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-1.760963,-0.183798,-0.912226,-0.184076,-1.749077
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,...,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.183798,-0.912226,0.436790,0.575758,-0.272573
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,...,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,-0.912226,0.436790,0.492543,-1.693875,-0.860182
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,...,ARIEL,3000.0,HC-ROPA LAVADO,HC-ROPA LAVADO-Liquido,HC-ROPA LAVADO-Liquido-ARIEL,0.436790,0.492543,-1.034841,2.470409,0.380806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006990,-0.006990,0.000000,0.000000
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,-0.006510,-0.006510,0.000000,0.000000
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005790,-0.005790,0.000000,0.000000
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,-0.005730,-0.005730,0.000000,0.000000


### Probamos invirtiendo la treandformacion

In [33]:
scalers = joblib.load('scalers_product_request_tn.joblib')
def inverse_scale_columns(df, columns_to_inverse_scale, scaler_dict):
    inverse_scaled_df = pd.DataFrame()

    for product_id, scaler in scaler_dict.items():
        # Check if the product_id is present in the original DataFrame
        if product_id in df['product_id'].unique() and scaler is not None:
            # Filter the original DataFrame for the current product_id
            group_data = df[df['product_id'] == product_id]

            # Extract the values to be inverse scaled for each column separately
            inverse_scaled_values = {}
            for column in columns_to_inverse_scale:
                values_to_inverse_scale = group_data[column].values.reshape(-1, 1)

                # Inverse scale the data
                inverse_scaled_column = scaler.inverse_transform(values_to_inverse_scale)

                # Store the inverse scaled column in the dictionary
                inverse_scaled_values[column] = inverse_scaled_column.flatten()

            # Create a DataFrame with the inverse scaled values and 'product_id' as the index
            group_df = pd.DataFrame(inverse_scaled_values, index=group_data.index)

            # Concatenate the group DataFrame to the overall inverse scaled DataFrame
            inverse_scaled_df = pd.concat([inverse_scaled_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_scaled_df = inverse_scaled_df.sort_index()

    return inverse_scaled_df

# Columns to inverse scale
columns_to_inverse_scale = ['cust_request_tn_scaled']

# Call the function to inverse scale the specified columns
inverse_scaled_columns_df = inverse_scale_columns(df_final_sellout_prod, columns_to_inverse_scale, scalers)

# Print the resulting DataFrame
print(inverse_scaled_columns_df)

       cust_request_tn_scaled
0                   937.72717
1                   833.72187
2                  1330.74697
3                  1132.94430
4                  1550.68936
...                       ...
21988                 0.00699
21989                 0.00651
21990                 0.00579
21991                 0.00573
21992                 0.00546

[21993 rows x 1 columns]


## Aca hacemos por categorias

In [34]:
cat_1=df_final_sellout_prod.groupby(['periodo','cat1'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_tn'})
cat_2=df_final_sellout_prod.groupby(['periodo','cat2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_tn'})
cat_3=df_final_sellout_prod.groupby(['periodo','cat3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_tn'})
brand=df_final_sellout_prod.groupby(['periodo','brand'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_tn'})
cat_1_2=df_final_sellout_prod.groupby(['periodo','cat_1_2'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_tn'})
cat_1_2_3=df_final_sellout_prod.groupby(['periodo','cat_1_2_3'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_tn'})
cat_1_2_3_brand=df_final_sellout_prod.groupby(['periodo','cat_1_2_3_brand'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_tn'})


In [35]:
df_final_sellout_prod = df_final_sellout_prod.merge(cat_1, on=['periodo', 'cat1'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(cat_2, on=['periodo', 'cat2'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(cat_3, on=['periodo', 'cat3'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(brand, on=['periodo', 'brand'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(cat_1_2, on=['periodo', 'cat_1_2'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(cat_1_2_3, on=['periodo', 'cat_1_2_3'], how='left')
df_final_sellout_prod = df_final_sellout_prod.merge(cat_1_2_3_brand, on=['periodo', 'cat_1_2_3_brand'], how='left')

In [36]:
cat_1_cust=df_final_sellout_prod_client.groupby(['periodo','cat1','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_cust_tn'})
cat_2_cust=df_final_sellout_prod_client.groupby(['periodo','cat2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_2_cust_tn'})
cat_3_cust=df_final_sellout_prod_client.groupby(['periodo','cat3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_3_cust_tn'})
brand_cust=df_final_sellout_prod_client.groupby(['periodo','brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'brand_cust_tn'})
cat_1_2_cust=df_final_sellout_prod_client.groupby(['periodo','cat_1_2','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_cust_tn'})
cat_1_2_3_cust=df_final_sellout_prod_client.groupby(['periodo','cat_1_2_3','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_cust_tn'})
cat_1_2_3_brand_cust=df_final_sellout_prod_client.groupby(['periodo','cat_1_2_3_brand','customer_id'])['tn'].sum().reset_index().rename(columns={'tn': 'cat_1_2_3_brand_cust_tn'})

In [37]:
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_cust, on=['periodo','customer_id', 'cat1'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_2_cust, on=['periodo', 'customer_id','cat2'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_3_cust, on=['periodo', 'customer_id','cat3'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(brand_cust, on=['periodo','customer_id', 'brand'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2_cust, on=['periodo','customer_id', 'cat_1_2'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2_3_cust, on=['periodo','customer_id', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2_3_brand_cust, on=['periodo','customer_id', 'cat_1_2_3_brand'], how='left')

In [38]:
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1, on=['periodo', 'cat1'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_2, on=['periodo','cat2'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_3, on=['periodo','cat3'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(brand, on=['periodo', 'brand'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2, on=['periodo', 'cat_1_2'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2_3, on=['periodo', 'cat_1_2_3'], how='left')
df_final_sellout_prod_client = df_final_sellout_prod_client.merge(cat_1_2_3_brand, on=['periodo' ,'cat_1_2_3_brand'], how='left')

In [39]:
df_final_sellout_prod.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'tn_mas_1', 'tn_mas_2', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand',
       'sku_size', 'cat_1_2', 'cat_1_2_3', 'cat_1_2_3_brand', 'tn_scaled',
       'tn_mas_1_scaled', 'tn_mas_2_scaled', 'cust_request_qty_scaled',
       'cust_request_tn_scaled', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn',
       'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn'],
      dtype='object')

In [40]:
## Columnas a estandarizar:
col_to_stand=['stock_final','cat_1_tn', 'cat_2_tn', 'cat_3_tn','brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn']

In [41]:
### Estandarizamos por las variables agregadas
for i in col_to_stand:
    # Define the column you want to standardize
    column_to_standardize = i

    # Group by 'product_id'
    grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

    # Initialize a dictionary to store scalers for each 'product_id'
    scalers = {}

    # Create an empty DataFrame to store the standardized values
    standardized_df = pd.DataFrame()

    # Standardize each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be standardized
        values_to_standardize = group_data.values.reshape(-1, 1)

        # Initialize and fit the scaler on the values without transforming
        scaler = StandardScaler()
        scaler.fit(values_to_standardize)

        # Save the scaler for this group in the dictionary
        scalers[group_name] = scaler

        # Transform the data
        standardized_values = scaler.transform(values_to_standardize)

        # Create a DataFrame with the standardized values and 'product_id' as the index
        group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall standardized DataFrame
        standardized_df = pd.concat([standardized_df, group_df])

    # Sort the DataFrame by index to get the original order
    standardized_df = standardized_df.sort_index()

    # Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

    # Save the scalers dictionary to a file
    import joblib

    joblib.dump(scalers, f'scalers_product_{i}.joblib')

    # ...

    # Load the scalers dictionary from the file
    scalers = joblib.load(f'scalers_product_{i}.joblib')

    # Create an empty DataFrame to store the inverse transformed values
    inverse_transformed_df = pd.DataFrame()

    # Inverse transform each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be inverse transformed
        values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

        # Retrieve the scaler for this group from the dictionary
        scaler = scalers[group_name]

        # Inverse transform the data using the original mean and standard deviation
        inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

        # Create a DataFrame with the inverse transformed values and 'product_id' as the index
        group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall inverse transformed DataFrame
        inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_transformed_df = inverse_transformed_df.sort_index()




    scalers = joblib.load(f'scalers_product_{i}.joblib')
    scalers

    def scale_columns(df, columns_to_scale, scaler_dict):
        scaled_df = pd.DataFrame()

        for product_id, scaler in scaler_dict.items():
            # Check if the product_id is present in the DataFrame
            if product_id in df['product_id'].unique() and scaler is not None:
                # Filter the DataFrame for the current product_id
                group_data = df[df['product_id'] == product_id]

                # Extract the values to be scaled for each column separately
                scaled_values = {}
                for column in columns_to_scale:
                    values_to_scale = group_data[column].values.reshape(-1, 1)
                    
                    # Scale the data
                    scaled_column = scaler.transform(values_to_scale)
                    
                    # Store the scaled column in the dictionary
                    scaled_values[column] = scaled_column.flatten()

                # Create a DataFrame with the scaled values and 'product_id' as the index
                group_df = pd.DataFrame(scaled_values, index=group_data.index)

                # Concatenate the group DataFrame to the overall scaled DataFrame
                scaled_df = pd.concat([scaled_df, group_df])

        # Sort the DataFrame by index to get the original order
        scaled_df = scaled_df.sort_index()

        return scaled_df

    # Example usage:
    # (Assuming you have already loaded the 'scalers' dictionary)

    # Columns to scale
    columns_to_scale = [f'{i}']

    # Call the function to scale the specified columns
    scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

    # Print the resulting DataFrame
    print(scaled_columns_df)


    # mergeamos las variables estandarizadas

    df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

    df_final_sellout_prod

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       stock_final
0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
...            ...
21988          NaN
21989          NaN
21990          NaN
21991          NaN
21992          NaN

[21993 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       cat_1_tn
0     -0.802880
1     -0.833825
2      1.391361
3      0.258303
4      1.507970
...         ...
21988       NaN
21989       NaN
21990       NaN
21991       NaN
21992       NaN

[21993 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

...

       cat_1_2_tn
0       -0.698108
1       -0.849425
2        1.186663
3       -0.135293
4        1.541501
...           ...
21988         NaN
21989         NaN
21990         NaN
21991         NaN
21992         NaN

[21993 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       cat_1_2_3_tn
0         -1.569261
1         -1.517277
2          0.479183
3         -1.070908
4          0.639177
...             ...
21988           NaN
21989           NaN
21990           NaN
21991           NaN
21992           NaN

[21993 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       cat_1_2_3_brand_tn
0               -1.414495
1               -1.726560
2                0.119643
3               -0.934709
4                0.366299
...                   ...
21988                 NaN
21989                 NaN
21990                 NaN
21991                 NaN
21992                 NaN

[21993 rows x 1 columns]


       cat_1_2_3_tn_scaled
0               3851.59182
1               3890.19286
2               5372.68487
3               4221.64889
4               5491.49052
...                    ...
21988                  NaN
21989                  NaN
21990                  NaN
21991                  NaN
21992                  NaN

[21993 rows x 1 columns]


# Creamos variable de Producto menor tamaño y mayor tamaño en los que tienen presentaciones similares

### Solo para productos   

In [42]:
prod_presen=df_final_sellout_prod.groupby(['periodo','cat_1_2_3_brand','sku_size'])['tn'].sum().reset_index()
prod_presen=prod_presen.sort_values(by=['cat_1_2_3_brand','periodo' ,'sku_size'])
# Sort the DataFrame by 'periodo' and 'cat_1_2_3_brand'
prod_presen = prod_presen.sort_values(by=['periodo', 'cat_1_2_3_brand'])

# Create a column 'tn_smaller_sku' for the 'tn' value when 'sku_size' is smaller
prod_presen['tn_smaller_sku'] = prod_presen.groupby(['periodo', 'cat_1_2_3_brand'])['tn'].shift(1)

# Create a column 'tn_bigger_sku' for the 'tn' value when 'sku_size' is bigger
prod_presen['tn_bigger_sku'] = prod_presen.groupby(['periodo', 'cat_1_2_3_brand'])['tn'].shift(-1)

# Print the resulting DataFrame

In [43]:
prod_presen

,periodo,cat_1_2_3_brand,sku_size,tn,tn_smaller_sku,tn_bigger_sku
0,201701,FOODS-ADEREZOS-Aji Picante-NATURA,240.0,6.32131,NaN,NaN
1,201701,FOODS-ADEREZOS-Barbacoa-NATURA,400.0,11.81983,NaN,NaN
2,201701,FOODS-ADEREZOS-Chimichurri-NATURA,350.0,6.03199,NaN,NaN
3,201701,FOODS-ADEREZOS-Ketchup-DEOS1,240.0,13.00026,NaN,NaN
4,201701,FOODS-ADEREZOS-Ketchup-MAYOS3,240.0,37.69038,NaN,NaN
...,...,...,...,...,...,...
7500,201902,REF-TE-Especial-TWININGS,10.0,0.02229,NaN,0.249
7501,201902,REF-TE-Especial-TWININGS,20.0,0.24900,0.02229,NaN
7502,201902,REF-TE-Frutas-TWININGS,20.0,0.02419,NaN,NaN
7503,201902,REF-TE-Hierbas-TWININGS,20.0,0.10045,NaN,NaN


In [44]:
# Validacion
prod_presen[prod_presen.cat_1_2_3_brand=='REF-TE-Especial-TWININGS']

,periodo,cat_1_2_3_brand,sku_size,tn,tn_smaller_sku,tn_bigger_sku
259,201701,REF-TE-Especial-TWININGS,10.0,0.33812,NaN,1.77932
260,201701,REF-TE-Especial-TWININGS,20.0,1.77932,0.33812,NaN
523,201702,REF-TE-Especial-TWININGS,10.0,0.13463,NaN,0.84022
524,201702,REF-TE-Especial-TWININGS,20.0,0.84022,0.13463,NaN
786,201703,REF-TE-Especial-TWININGS,10.0,0.47365,NaN,0.45727
787,201703,REF-TE-Especial-TWININGS,20.0,0.45727,0.47365,NaN
1052,201704,REF-TE-Especial-TWININGS,10.0,0.01966,NaN,0.69449
1053,201704,REF-TE-Especial-TWININGS,20.0,0.69449,0.01966,NaN
1319,201705,REF-TE-Especial-TWININGS,10.0,0.02555,NaN,0.73968
1320,201705,REF-TE-Especial-TWININGS,20.0,0.73968,0.02555,NaN


### Para productos y clientes 

In [45]:
prod_presen_cust=df_final_sellout_prod_client.groupby(['periodo','customer_id','cat_1_2_3_brand','sku_size'])['tn'].sum().reset_index()
prod_presen_cust=prod_presen_cust.sort_values(by=['cat_1_2_3_brand','periodo' ,'sku_size'])
# Sort the DataFrame by 'periodo' and 'cat_1_2_3_brand'
prod_presen_cust = prod_presen_cust.sort_values(by=['customer_id','periodo', 'cat_1_2_3_brand'])
# Create a column 'tn_smaller_sku' for the 'tn' value when 'sku_size' is smaller
prod_presen_cust['tn_smaller_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(1)

# Create a column 'tn_bigger_sku' for the 'tn' value when 'sku_size' is bigger
prod_presen_cust['tn_bigger_sku_cust'] = prod_presen_cust.groupby(['customer_id','periodo', 'cat_1_2_3_brand'])['tn'].shift(-1)


In [46]:
prod_presen_cust[~prod_presen_cust.tn_smaller_sku_cust.isna()]

,periodo,customer_id,cat_1_2_3_brand,sku_size,tn,tn_smaller_sku_cust,tn_bigger_sku_cust
6,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,375.0,0.99590,10.92000,3.45946
7,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,400.0,3.45946,0.99590,0.65520
8,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,500.0,0.65520,3.45946,NaN
10,201701,10001,FOODS-ADEREZOS-Mayonesa-DEOS1,340.0,0.51456,2.81970,2.37667
11,201701,10001,FOODS-ADEREZOS-Mayonesa-DEOS1,475.0,2.37667,0.51456,NaN
...,...,...,...,...,...,...,...
33604,201701,10635,PC-DEOS-Pies Talco-DEOS1,200.0,0.01747,0.01310,NaN
107113,201703,10635,PC-CABELLO-Styling-GOMINA,280.0,0.00153,0.00328,NaN
107115,201703,10635,PC-DEOS-Aero-COLBERT,100.0,0.00630,0.00392,NaN
107118,201703,10635,PC-DEOS-Aero-NIVEA,100.0,0.00109,0.00194,NaN


In [47]:
# Validacion
prod_presen_cust[(prod_presen_cust.customer_id==10001) & (prod_presen_cust.cat_1_2_3_brand=='FOODS-ADEREZOS-Ketchup-NATURA') ]

,periodo,customer_id,cat_1_2_3_brand,sku_size,tn,tn_smaller_sku_cust,tn_bigger_sku_cust
5,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,240.0,10.92000,NaN,0.99590
6,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,375.0,0.99590,10.92000,3.45946
7,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,400.0,3.45946,0.99590,0.65520
8,201701,10001,FOODS-ADEREZOS-Ketchup-NATURA,500.0,0.65520,3.45946,NaN
33614,201702,10001,FOODS-ADEREZOS-Ketchup-NATURA,240.0,20.96640,NaN,0.66394
...,...,...,...,...,...,...,...
889265,201901,10001,FOODS-ADEREZOS-Ketchup-NATURA,500.0,4.58640,2.59459,NaN
919522,201902,10001,FOODS-ADEREZOS-Ketchup-NATURA,240.0,8.29920,NaN,0.65564
919523,201902,10001,FOODS-ADEREZOS-Ketchup-NATURA,375.0,0.65564,8.29920,2.20147
919524,201902,10001,FOODS-ADEREZOS-Ketchup-NATURA,400.0,2.20147,0.65564,10.59240



### Joineamos con las bases totales




In [48]:
df_final_sellout_prod=pd.merge(df_final_sellout_prod, prod_presen[['periodo', 'cat_1_2_3_brand', 'sku_size', 'tn_smaller_sku', 'tn_bigger_sku']], on=['periodo', 'cat_1_2_3_brand','sku_size'], how='left')


In [49]:
df_final_sellout_prod_client=pd.merge(df_final_sellout_prod_client, prod_presen[['periodo', 'cat_1_2_3_brand', 'sku_size', 'tn_smaller_sku', 'tn_bigger_sku']], on=['periodo', 'cat_1_2_3_brand','sku_size'], how='left')
df_final_sellout_prod_client=pd.merge(df_final_sellout_prod_client, prod_presen_cust[['periodo','customer_id', 'cat_1_2_3_brand', 'sku_size', 'tn_smaller_sku_cust', 'tn_bigger_sku_cust']], on=['periodo','customer_id', 'cat_1_2_3_brand','sku_size'], how='left')


In [50]:
col_to_stand=['tn_smaller_sku', 'tn_bigger_sku']
### Estandarizamos por cat_1_tn
for i in col_to_stand:
    # Define the column you want to standardize
    column_to_standardize = i

    # Group by 'product_id'
    grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

    # Initialize a dictionary to store scalers for each 'product_id'
    scalers = {}

    # Create an empty DataFrame to store the standardized values
    standardized_df = pd.DataFrame()

    # Standardize each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be standardized
        values_to_standardize = group_data.values.reshape(-1, 1)

        # Initialize and fit the scaler on the values without transforming
        scaler = StandardScaler()
        scaler.fit(values_to_standardize)

        # Save the scaler for this group in the dictionary
        scalers[group_name] = scaler

        # Transform the data
        standardized_values = scaler.transform(values_to_standardize)

        # Create a DataFrame with the standardized values and 'product_id' as the index
        group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall standardized DataFrame
        standardized_df = pd.concat([standardized_df, group_df])

    # Sort the DataFrame by index to get the original order
    standardized_df = standardized_df.sort_index()

    # Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

    # Save the scalers dictionary to a file
    import joblib

    joblib.dump(scalers, f'scalers_product_{i}.joblib')

    # ...

    # Load the scalers dictionary from the file
    scalers = joblib.load(f'scalers_product_{i}.joblib')

    # Create an empty DataFrame to store the inverse transformed values
    inverse_transformed_df = pd.DataFrame()

    # Inverse transform each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be inverse transformed
        values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

        # Retrieve the scaler for this group from the dictionary
        scaler = scalers[group_name]

        # Inverse transform the data using the original mean and standard deviation
        inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

        # Create a DataFrame with the inverse transformed values and 'product_id' as the index
        group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall inverse transformed DataFrame
        inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_transformed_df = inverse_transformed_df.sort_index()




    scalers = joblib.load(f'scalers_product_{i}.joblib')
    scalers

    def scale_columns(df, columns_to_scale, scaler_dict):
        scaled_df = pd.DataFrame()

        for product_id, scaler in scaler_dict.items():
            # Check if the product_id is present in the DataFrame
            if product_id in df['product_id'].unique() and scaler is not None:
                # Filter the DataFrame for the current product_id
                group_data = df[df['product_id'] == product_id]

                # Extract the values to be scaled for each column separately
                scaled_values = {}
                for column in columns_to_scale:
                    values_to_scale = group_data[column].values.reshape(-1, 1)
                    
                    # Scale the data
                    scaled_column = scaler.transform(values_to_scale)
                    
                    # Store the scaled column in the dictionary
                    scaled_values[column] = scaled_column.flatten()

                # Create a DataFrame with the scaled values and 'product_id' as the index
                group_df = pd.DataFrame(scaled_values, index=group_data.index)

                # Concatenate the group DataFrame to the overall scaled DataFrame
                scaled_df = pd.concat([scaled_df, group_df])

        # Sort the DataFrame by index to get the original order
        scaled_df = scaled_df.sort_index()

        return scaled_df

    # Example usage:
    # (Assuming you have already loaded the 'scalers' dictionary)

    # Columns to scale
    columns_to_scale = [f'{i}']

    # Call the function to scale the specified columns
    scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

    # Print the resulting DataFrame
    print(scaled_columns_df)


    # mergeamos las variables estandarizadas

    df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

    df_final_sellout_prod

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       tn_smaller_sku
0            0.503587
1            0.680864
2            1.946621
3            0.485934
4            0.725477
...               ...
21988             NaN
21989             NaN
21990             NaN
21991             NaN
21992             NaN

[21993 rows x 1 columns]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: inv

       tn_bigger_sku
0                NaN
1                NaN
2                NaN
3          -0.555630
4          -0.116324
...              ...
21988            NaN
21989            NaN
21990            NaN
21991            NaN
21992            NaN

[21993 rows x 1 columns]


In [51]:
df_final_sellout_prod

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,...,cat_2_tn_scaled,cat_3_tn_scaled,brand_tn_scaled,cat_1_2_tn_scaled,cat_1_2_3_tn_scaled,cat_1_2_3_brand_tn_scaled,tn_smaller_sku,tn_bigger_sku,tn_smaller_sku_scaled,tn_bigger_sku_scaled
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,...,-0.698108,-1.568866,-0.885958,-0.698108,-1.569261,-1.414495,247.07431,NaN,0.503587,NaN
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,...,-0.849425,-1.518514,-1.657720,-0.849425,-1.517277,-1.726560,265.62114,NaN,0.680864,NaN
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,...,1.186663,0.491410,0.589231,1.186663,0.479183,0.119643,398.04551,NaN,1.946621,NaN
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,...,-0.135293,-1.069182,-0.537252,-0.135293,-1.070908,-0.934709,245.22745,14.47628,0.485934,-0.555630
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,...,1.541501,0.648404,1.447591,1.541501,0.639177,0.366299,270.28857,23.74408,0.725477,-0.116324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Agregamos de fuentes externas el valor del dolar, ipc, TCRM, tasa de interes de caja de ahorro y plazo fijo y la temperatura maxima y minima para buenos aires

In [52]:
dolar=pd.read_csv('dolar_data.csv')
tempe=pd.read_csv('temperatura.csv')


dolar.periodo.astype(int)
tempe.periodo.astype(int)

0     201701
1     201702
2     201703
3     201704
4     201705
5     201706
6     201707
7     201708
8     201709
9     201710
10    201711
11    201712
12    201801
13    201802
14    201803
15    201804
16    201805
17    201806
18    201807
19    201808
20    201809
21    201810
22    201811
23    201812
24    201901
25    201902
26    201903
27    201904
28    201905
29    201906
30    201907
31    201908
32    201909
33    201910
34    201911
35    201912
36    202001
37    202002
38    202003
39    202004
40    202005
41    202006
42    202007
43    202008
44    202009
45    202010
46    202011
47    202012
48    202101
49    202102
50    202103
51    202104
52    202105
53    202106
54    202107
55    202108
56    202109
57    202110
58    202111
59    202112
Name: periodo, dtype: int64

In [53]:
df_final_sellout_prod=pd.merge(df_final_sellout_prod, dolar, on='periodo', how='left')
df_final_sellout_prod_client=pd.merge(df_final_sellout_prod_client, dolar, on='periodo', how='left')

In [54]:
df_final_sellout_prod=pd.merge(df_final_sellout_prod, tempe, on='periodo', how='left')
df_final_sellout_prod_client=pd.merge(df_final_sellout_prod_client, tempe, on='periodo', how='left')

In [55]:
df_final_sellout_prod.columns

Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn', 'tn',
       'tn_mas_1', 'tn_mas_2', 'stock_final', 'cat1', 'cat2', 'cat3', 'brand',
       'sku_size', 'cat_1_2', 'cat_1_2_3', 'cat_1_2_3_brand', 'tn_scaled',
       'tn_mas_1_scaled', 'tn_mas_2_scaled', 'cust_request_qty_scaled',
       'cust_request_tn_scaled', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn',
       'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn',
       'stock_final_scaled', 'cat_1_tn_scaled', 'cat_2_tn_scaled',
       'cat_3_tn_scaled', 'brand_tn_scaled', 'cat_1_2_tn_scaled',
       'cat_1_2_3_tn_scaled', 'cat_1_2_3_brand_tn_scaled', 'tn_smaller_sku',
       'tn_bigger_sku', 'tn_smaller_sku_scaled', 'tn_bigger_sku_scaled',
       'usd_blue_max', 'usd_oficial_max', 'ipc', 'itcrm', 'int_ca', 'int_pf',
       'temp_max', 'temp_min'],
      dtype='object')

### Estandarizamos las variables

In [ ]:
col_to_stand=['usd_blue_max', 'usd_oficial_max', 'ipc', 'itcrm', 'int_ca', 'int_pf','temp_max', 'temp_min']
### Estandarizamos por cat_1_tn
for i in col_to_stand:
    # Define the column you want to standardize
    column_to_standardize = i

    # Group by 'product_id'
    grouped = df_final_sellout_prod.groupby('product_id')[column_to_standardize]

    # Initialize a dictionary to store scalers for each 'product_id'
    scalers = {}

    # Create an empty DataFrame to store the standardized values
    standardized_df = pd.DataFrame()

    # Standardize each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be standardized
        values_to_standardize = group_data.values.reshape(-1, 1)

        # Initialize and fit the scaler on the values without transforming
        scaler = StandardScaler()
        scaler.fit(values_to_standardize)

        # Save the scaler for this group in the dictionary
        scalers[group_name] = scaler

        # Transform the data
        standardized_values = scaler.transform(values_to_standardize)

        # Create a DataFrame with the standardized values and 'product_id' as the index
        group_df = pd.DataFrame(standardized_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall standardized DataFrame
        standardized_df = pd.concat([standardized_df, group_df])

    # Sort the DataFrame by index to get the original order
    standardized_df = standardized_df.sort_index()

    # Now, 'standardized_df' contains the standardized values for 'tn' grouped by 'product_id'

    # Save the scalers dictionary to a file
    import joblib

    joblib.dump(scalers, f'scalers_product_{i}.joblib')

    # ...

    # Load the scalers dictionary from the file
    scalers = joblib.load(f'scalers_product_{i}.joblib')

    # Create an empty DataFrame to store the inverse transformed values
    inverse_transformed_df = pd.DataFrame()

    # Inverse transform each group and concatenate the results
    for group_name, group_data in grouped:
        # Extract the values to be inverse transformed
        values_to_inverse_transform = standardized_df.loc[group_data.index][column_to_standardize].values.reshape(-1, 1)

        # Retrieve the scaler for this group from the dictionary
        scaler = scalers[group_name]

        # Inverse transform the data using the original mean and standard deviation
        inverse_transformed_values = scaler.inverse_transform(values_to_inverse_transform)

        # Create a DataFrame with the inverse transformed values and 'product_id' as the index
        group_df = pd.DataFrame(inverse_transformed_values, index=group_data.index, columns=[column_to_standardize])

        # Concatenate the group DataFrame to the overall inverse transformed DataFrame
        inverse_transformed_df = pd.concat([inverse_transformed_df, group_df])

    # Sort the DataFrame by index to get the original order
    inverse_transformed_df = inverse_transformed_df.sort_index()




    scalers = joblib.load(f'scalers_product_{i}.joblib')
    scalers

    def scale_columns(df, columns_to_scale, scaler_dict):
        scaled_df = pd.DataFrame()

        for product_id, scaler in scaler_dict.items():
            # Check if the product_id is present in the DataFrame
            if product_id in df['product_id'].unique() and scaler is not None:
                # Filter the DataFrame for the current product_id
                group_data = df[df['product_id'] == product_id]

                # Extract the values to be scaled for each column separately
                scaled_values = {}
                for column in columns_to_scale:
                    values_to_scale = group_data[column].values.reshape(-1, 1)
                    
                    # Scale the data
                    scaled_column = scaler.transform(values_to_scale)
                    
                    # Store the scaled column in the dictionary
                    scaled_values[column] = scaled_column.flatten()

                # Create a DataFrame with the scaled values and 'product_id' as the index
                group_df = pd.DataFrame(scaled_values, index=group_data.index)

                # Concatenate the group DataFrame to the overall scaled DataFrame
                scaled_df = pd.concat([scaled_df, group_df])

        # Sort the DataFrame by index to get the original order
        scaled_df = scaled_df.sort_index()

        return scaled_df

    # Example usage:
    # (Assuming you have already loaded the 'scalers' dictionary)

    # Columns to scale
    columns_to_scale = [f'{i}']

    # Call the function to scale the specified columns
    scaled_columns_df = scale_columns(df_final_sellout_prod, columns_to_scale, scalers)

    # Print the resulting DataFrame
    print(scaled_columns_df)


    # mergeamos las variables estandarizadas

    df_final_sellout_prod = pd.merge(df_final_sellout_prod, scaled_columns_df, left_index=True, right_index=True, suffixes=('', '_scaled'))

    df_final_sellout_prod

# Empezamos con el FE temporal 

## Considerando solo Product_id y periodo

El código proporcionado define una función llamada create_lagged_features que se utiliza para crear características de retraso, estadísticas móviles, características de tendencia y características de estacionariedad en un DataFrame de pandas. Estas características son útiles para modelar series temporales y otros tipos de datos secuenciales.

La función toma cuatro parámetros: df que es el DataFrame de entrada, lag_periods que es una lista de los períodos de retraso a considerar, window_sizes que es una lista de los tamaños de ventana para las estadísticas móviles, y feature_names que es una lista de los nombres de las características para las que se crearán las nuevas características.

Primero, la función ordena el DataFrame por 'product_id' y 'periodo'. Luego, para cada nombre de característica en feature_names, para cada tamaño de ventana en window_sizes, y para cada período de retraso en lag_periods, la función crea una nueva característica de retraso. Esto se hace utilizando el método shift de pandas, que desplaza los datos hacia abajo en un número especificado de filas (el período de retraso).

Después, para cada nombre de característica y cada tamaño de ventana, la función crea tres nuevas características: una media móvil, una desviación estándar móvil y una característica de tendencia. La media móvil y la desviación estándar móvil se calculan utilizando el método rolling de pandas, que proporciona una ventana móvil de los datos. La característica de tendencia se calcula utilizando el método diff de pandas, que calcula la diferencia entre filas consecutivas.

Finalmente, la función crea características de estacionariedad adicionales utilizando más características de retraso (hasta 24 períodos). Estas características se crean de la misma manera que las características de retraso anteriores.

La función devuelve el DataFrame con las características añadidas.

In [57]:
def create_lagged_features(df, lag_periods, window_sizes, feature_names):
    """
    Create lagged features, rolling statistics, trend features, and stationarity features for the specified feature names.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    lag_periods (list): A list of lag periods to consider.
    window_sizes (list): A list of window sizes for rolling statistics.
    feature_names (list): A list of feature names to create features for.

    Returns:
    pd.DataFrame: A DataFrame with the added features.
    """
    df = df.sort_values(by=['product_id', 'periodo'])

    for feature_name in feature_names:
        for lag in lag_periods:
            df[f'{feature_name}_lag_{lag}'] = df.groupby('product_id')[feature_name].shift(lag)
            df[f'{feature_name}_trend_lag_{lag}'] = df.groupby('product_id')[feature_name].diff(lag)
                        # Additional features for each lag period
            df[f'{feature_name}_lag_{lag}_ratio'] = df.groupby('product_id')[feature_name].pct_change().shift(lag)
            df[f'{feature_name}_lag_{lag}_cumulative_sum'] = df.groupby('product_id')[feature_name].cumsum().shift(lag)



        for window_size in window_sizes:
            df[f'{feature_name}_rolling_mean_win_{window_size}'] = df.groupby('product_id')[feature_name].rolling(window=window_size).mean().reset_index(level=0, drop=True)
            df[f'{feature_name}_rolling_std_win_{window_size}'] = df.groupby('product_id')[feature_name].rolling(window=window_size).std().reset_index(level=0, drop=True)


    return df

In [58]:
df_final_sellout_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21993 entries, 0 to 21992
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   periodo                    21993 non-null  int64  
 1   product_id                 21993 non-null  int64  
 2   cust_request_qty           21993 non-null  float64
 3   cust_request_tn            21993 non-null  float64
 4   tn                         21993 non-null  float64
 5   tn_mas_1                   21993 non-null  float64
 6   tn_mas_2                   21993 non-null  float64
 7   stock_final                4461 non-null   float64
 8   cat1                       21781 non-null  object 
 9   cat2                       21781 non-null  object 
 10  cat3                       21781 non-null  object 
 11  brand                      21781 non-null  object 
 12  sku_size                   21781 non-null  float64
 13  cat_1_2                    21781 non-null  obj

In [60]:
numeric_cols = df_final_sellout_prod.select_dtypes(include='float').columns.tolist()
print(numeric_cols)
numeric_cols=[ 'cust_request_qty', 'cust_request_tn', 'tn', 'stock_final', 'sku_size', 'tn_scaled', 
               'cust_request_qty_scaled', 'cust_request_tn_scaled', 'cat_1_tn', 'cat_2_tn', 
              'cat_3_tn', 'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn', 'stock_final_scaled', 'cat_1_tn_scaled', 
              'cat_2_tn_scaled', 'cat_3_tn_scaled', 'brand_tn_scaled', 'cat_1_2_tn_scaled', 'cat_1_2_3_tn_scaled', 'cat_1_2_3_brand_tn_scaled',
                'tn_smaller_sku', 'tn_bigger_sku', 'tn_smaller_sku_scaled', 'tn_bigger_sku_scaled', 'usd_blue_max', 'usd_oficial_max', 'ipc', 'itcrm', 
                'int_ca', 'int_pf', 'temp_max', 'temp_min', 'usd_blue_max_scaled', 'usd_oficial_max_scaled', 'ipc_scaled', 'itcrm_scaled', 'int_ca_scaled',
                  'int_pf_scaled', 'temp_max_scaled', 'temp_min_scaled']




['cust_request_qty', 'cust_request_tn', 'tn', 'tn_mas_1', 'tn_mas_2', 'stock_final', 'sku_size', 'tn_scaled', 'tn_mas_1_scaled', 'tn_mas_2_scaled', 'cust_request_qty_scaled', 'cust_request_tn_scaled', 'cat_1_tn', 'cat_2_tn', 'cat_3_tn', 'brand_tn', 'cat_1_2_tn', 'cat_1_2_3_tn', 'cat_1_2_3_brand_tn', 'stock_final_scaled', 'cat_1_tn_scaled', 'cat_2_tn_scaled', 'cat_3_tn_scaled', 'brand_tn_scaled', 'cat_1_2_tn_scaled', 'cat_1_2_3_tn_scaled', 'cat_1_2_3_brand_tn_scaled', 'tn_smaller_sku', 'tn_bigger_sku', 'tn_smaller_sku_scaled', 'tn_bigger_sku_scaled', 'usd_blue_max', 'usd_oficial_max', 'ipc', 'itcrm', 'int_ca', 'int_pf', 'temp_max', 'temp_min', 'usd_blue_max_scaled', 'usd_oficial_max_scaled', 'ipc_scaled', 'itcrm_scaled', 'int_ca_scaled', 'int_pf_scaled', 'temp_max_scaled', 'temp_min_scaled']


In [61]:
numeric_cols

['cust_request_qty',
 'cust_request_tn',
 'tn',
 'stock_final',
 'sku_size',
 'tn_scaled',
 'cust_request_qty_scaled',
 'cust_request_tn_scaled',
 'cat_1_tn',
 'cat_2_tn',
 'cat_3_tn',
 'brand_tn',
 'cat_1_2_tn',
 'cat_1_2_3_tn',
 'cat_1_2_3_brand_tn',
 'stock_final_scaled',
 'cat_1_tn_scaled',
 'cat_2_tn_scaled',
 'cat_3_tn_scaled',
 'brand_tn_scaled',
 'cat_1_2_tn_scaled',
 'cat_1_2_3_tn_scaled',
 'cat_1_2_3_brand_tn_scaled',
 'tn_smaller_sku',
 'tn_bigger_sku',
 'tn_smaller_sku_scaled',
 'tn_bigger_sku_scaled',
 'usd_blue_max',
 'usd_oficial_max',
 'ipc',
 'itcrm',
 'int_ca',
 'int_pf',
 'temp_max',
 'temp_min',
 'usd_blue_max_scaled',
 'usd_oficial_max_scaled',
 'ipc_scaled',
 'itcrm_scaled',
 'int_ca_scaled',
 'int_pf_scaled',
 'temp_max_scaled',
 'temp_min_scaled']

In [ ]:

lag_periods = range(1, 25)  # Consider lags from 1 to 24 periods
window_sizes = range(1, 13)  # Consider window sizes from 1 to 12
feature_names = numeric_cols  # Add feature names you want to process

# Create lagged features, rolling statistics, trends, and stationarity features for the specified features
df_final_sellout_prod_fe = create_lagged_features(df_final_sellout_prod, lag_periods, window_sizes, feature_names)

# Select the features you want for your model
#selected_features = [col for col in df.columns if any(feat in col for feat in feature_names)]

# Now you can use the df DataFrame for training your model with the selected features

In [63]:

na_counts = df_final_sellout_prod_fe.isna().sum()
print(na_counts)


periodo                                    0
product_id                                 0
cust_request_qty                           0
cust_request_tn                            0
tn                                         0
                                       ...  
temp_min_scaled_rolling_std_win_10      8905
temp_min_scaled_rolling_mean_win_11     9808
temp_min_scaled_rolling_std_win_11      9808
temp_min_scaled_rolling_mean_win_12    10691
temp_min_scaled_rolling_std_win_12     10691
Length: 5216, dtype: int64


In [69]:
df_final_sellout_prod_fe

,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,cat2,...,temp_min_scaled_rolling_mean_win_8,temp_min_scaled_rolling_std_win_8,temp_min_scaled_rolling_mean_win_9,temp_min_scaled_rolling_std_win_9,temp_min_scaled_rolling_mean_win_10,temp_min_scaled_rolling_std_win_10,temp_min_scaled_rolling_mean_win_11,temp_min_scaled_rolling_std_win_11,temp_min_scaled_rolling_mean_win_12,temp_min_scaled_rolling_std_win_12
0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,ROPA LAVADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,ROPA LAVADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,ROPA LAVADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,ROPA LAVADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,ROPA LAVADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
#Agregamos el indice para poder coordinar posteriormente con el scaler
df_final_sellout_prod_fe['id_row']=df_final_sellout_prod_fe.index

/var/folders/5_/gws729vx0pn71fdvj09_cqghn9zl4w/T/ipykernel_37911/1500278302.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_sellout_prod_fe['id_row']=df_final_sellout_prod_fe.index


In [73]:
columns_order = ['id_row'] + [col for col in df_final_sellout_prod_fe.columns if col != 'id_row']
df_final_sellout_prod_fe = df_final_sellout_prod_fe[columns_order]

In [74]:
df_final_sellout_prod_fe

,id_row,periodo,product_id,cust_request_qty,cust_request_tn,tn,tn_mas_1,tn_mas_2,stock_final,cat1,...,temp_min_scaled_rolling_mean_win_8,temp_min_scaled_rolling_std_win_8,temp_min_scaled_rolling_mean_win_9,temp_min_scaled_rolling_std_win_9,temp_min_scaled_rolling_mean_win_10,temp_min_scaled_rolling_std_win_10,temp_min_scaled_rolling_mean_win_11,temp_min_scaled_rolling_std_win_11,temp_min_scaled_rolling_mean_win_12,temp_min_scaled_rolling_std_win_12
0,0,201701,20001,479.0,937.72717,934.77222,798.01620,1303.35771,NaN,HC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,201702,20001,432.0,833.72187,798.01620,1303.35771,1069.96130,NaN,HC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,201703,20001,509.0,1330.74697,1303.35771,1069.96130,1502.20132,NaN,HC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,201704,20001,279.0,1132.94430,1069.96130,1502.20132,1520.06539,NaN,HC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,201705,20001,701.0,1550.68936,1502.20132,1520.06539,1030.67391,NaN,HC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21988,21988,201701,21295,1.0,0.00699,0.00699,0.00000,0.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21989,21989,201708,21296,1.0,0.00651,0.00651,0.00000,0.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21990,21990,201701,21297,1.0,0.00579,0.00579,0.00000,0.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21991,21991,201708,21298,1.0,0.00573,0.00573,0.00000,0.00000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df_final_sellout_prod_fe.to_parquet('df_final_sellout_prod_fe.parquet', index=False)
